In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from datetime import datetime

client = Elasticsearch(["http://127.0.0.1:9200"])

INDEX = "git_opnfv"

Total de organizaciones involucradas:

In [ ]:
s = Search(using=client, index=INDEX)
s.aggs.metric('companies', 'cardinality', field='author_org_name')
result = s.execute()
result.to_dict()['aggregations']["companies"]["value"]

Top 5 organizaciones involucradas:

In [ ]:
s = Search(using=client, index=INDEX)
s.aggs.bucket('by_company', 'terms', field='author_org_name', size=5, order={'commits':'desc'}).metric('commits', 'cardinality', field='hash')
result = s.execute()
result.to_dict()['aggregations']

for company in result.to_dict()['aggregations']["by_company"]["buckets"]:
    print("company: " + company["key"])
    print('  commits: ' + str(company['commits']['value']))

Numero total de autores:

In [ ]:
s = Search(using=client, index=INDEX)
s.aggs.metric('authors', 'cardinality', field='author_name')
result = s.execute()
result.to_dict()['aggregations']["authors"]["value"]

Top 5 autores:

In [ ]:
s = Search(using=client, index=INDEX)
s.aggs.bucket('by_author', 'terms', field='author_name', size=5, order={'commits':'desc'}).metric('commits', 'cardinality', field='hash')
result = s.execute()
result.to_dict()['aggregations']

for author in result.to_dict()['aggregations']["by_author"]["buckets"]:
    print("author: " + author["key"])
    print('  commits: ' + str(author['commits']['value']))

Número total de repositorios:

In [ ]:
s = Search(using=client, index=INDEX)
s.aggs.metric('repos', 'cardinality', field='repo_name')
result = s.execute()
result.to_dict()['aggregations']["repos"]["value"]

Top 5 repositorios:

In [ ]:
s = Search(using=client, index=INDEX)
s.aggs.bucket('by_repo', 'terms', field='repo_name', size=5, order={'commits':'desc'}).metric('commits', 'cardinality', field='hash')
result = s.execute()
result.to_dict()['aggregations']

for repo in result.to_dict()['aggregations']["by_repo"]["buckets"]:
    print("repo: " + repo["key"])
    print('  commits: ' + str(repo['commits']['value']))

De las 5 compañias mas importantes, sacar los 3 autores que más commits tienen con los 3 repos a los que mas contribuyen:

In [ ]:
s = Search(using=client, index=INDEX)
s.aggs.bucket('by_organization', 'terms', field='author_org_name',size=3, order={'commits':'desc'})\
.metric('commits', 'cardinality', field='hash')\
.bucket('by_author', 'terms', field='author_name', size=3, order={'commits':'desc'})\
.metric('commits', 'cardinality', field='hash')\
.bucket('by_repo', 'terms', field='repo_name', size=2, order={'commits':'desc'})\
.metric('commits', 'cardinality', field='hash')
result = s.execute()
result.to_dict()["aggregations"]

for key in result.to_dict()["aggregations"]["by_organization"]["buckets"]:
    print("company: " + key["key"])
    for author in key["by_author"]["buckets"]:
        print("  author: " + author["key"])
        for repo in author["by_repo"]["buckets"]: 
            print("    repo: " + repo["key"])

Autores que han trabajado para mas de una compañía

In [ ]:
s = Search(using=client, index=INDEX)
s.aggs.bucket('by_author', 'terms', field='author_name', size=700)\
.bucket('by_org', 'terms', field='author_org_name')\
.metric('companies', 'cardinality', field='author_org_name')
result = s.execute()
orgs = result.to_dict()["aggregations"]["by_author"]["buckets"]
for org in orgs:
    if len(org["by_org"]["buckets"])!=1:
        print("author: " + org['key'])
        for company in org["by_org"]["buckets"]:
            print("  company: " + company["key"])